<a href="https://colab.research.google.com/github/kbrezinski/CS224W-GraphML/blob/main/notebooks/practice-GNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.__version__)

from platform import python_version
print(python_version())

1.11.0+cu113
3.7.13


In [2]:
!pip install torch-scatter torch-sparse \
 torch-cluster torch-spline-conv torch-geometric \
-f https://data.pyg.org/whl/torch-1.11.0+cu113.html -q
#!pip install ogb

     |████████████████████████████████| 7.9 MB 6.8 MB/s 
     |████████████████████████████████| 3.5 MB 70.9 MB/s 
     |████████████████████████████████| 2.5 MB 9.2 MB/s 
     |████████████████████████████████| 750 kB 51.2 MB/s 
     |████████████████████████████████| 407 kB 6.1 MB/s 


In [45]:
import os
import torch_geometric.transforms as T

from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader, NeighborLoader

In [36]:
dataset = 'Cora'
transform = T.Compose([
    T.RandomNodeSplit('train_rest', num_val=500, num_test=500),
    T.TargetIndegree(),
    T.NormalizeFeatures(),
])
path = os.path.join('data', dataset)
dataset = Planetoid(path, dataset, transform=transform)  # transform is post download

In [61]:
# need to seperate loader for train and validate
loader = NeighborLoader(dataset[0], num_neighbors=[80]*3)

In [1]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GCNConv(dataset.num_node_features, 16)
    self.conv2 = GCNConv(16, dataset.num_classes)

  def forward(self, data):
    x, edge_index = data.x, data.edge_index

    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = F.dropout(x, training=self.training)
    x = self.conv2(x, edge_index)

    return F.log_softmax(x, dim=1)

ModuleNotFoundError: ignored

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-4)

model.train()
for epoch in range(200):
  for batch in loader:
    data = batch.to(device)
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[batch.train_mask], data.y[batch.train_mask])
    loss.backward()
    optimizer.step()

  if not epoch % 20:
    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    print(f'Accuracy: {acc:.4f}')